In [5]:
# -*- coding: utf-8 -*-
"""
Procesamiento de la hoja "Carozos" del archivo Excel "MAESTRO CAROZOS FINAL COMPLETO CG.xlsx".

El script:
1. Lee las filas desde la 3 (A3) hasta la última y las columnas de A a AP.
2. Filtra únicamente las filas donde la columna **Especie** sea "Ciruela" o "Nectarin".
3. Para "Ciruela":
   - Selecciona la sección "FIRMEZA PUNTO DEBIL – MODA INF" → columnas: *Quilla*, *Hombro*, *Mejilla 1*, *Mejilla 2*.
   - Toma los valores de *Solidos solubles (%)* como **BRIX**.
   - Mantiene la columna *Acidez (%)*.
4. Renombra *Solidos solubles (%)* → **BRIX** para que el DataFrame sea homogéneo.
5. Convierte **Fecha cosecha** a formato fecha y, si está vacía, rellena con el promedio de las fechas disponibles.
6. Calcula la suma de las columnas de condiciones (Quilla, Hombro, Mejilla 1, Mejilla 2, BRIX, Acidez (%)).
7. Genera 4 clústeres (cuartiles) a partir de esa suma y agrega una nueva columna **cluster** rotulada 1‑4.

Uso:
>>> python procesar_carozos.py

El DataFrame final se imprime en pantalla y puede reutilizarse desde otras
funciones importando `process_carozos`.
"""

import os
from pathlib import Path
from typing import Union, List

import numpy as np
import pandas as pd

# --------------------------------------------------------------------------------------
# Configuración
# --------------------------------------------------------------------------------------
path = r"C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\MAESTRO CAROZOS FINAL COMPLETO CG.xlsx"
SHEET_NAME = "CAROZOS"
USECOLS = "A:AP"  # Columnas A … AP
START_ROW = 2       # Cero‑indexado — saltamos las dos primeras filas para que A3 sea la primera

# Columnas a mantener
COLUMNS_FIRMEZA = [
    "Quilla",
    "Hombro",
    "Mejilla 1",
    "Mejilla 2",
]
COLUMNS_CALIDAD = [
    "Solidos solubles (%)",  # se renombrará a BRIX
    "Acidez (%)",
]
DATE_COLUMN = "Fecha cosecha"
ESPECIE_COLUMN = "Especie"
ESPECIES_VALIDAS = {"Ciruela", "Nectarin"}

# --------------------------------------------------------------------------------------
# Funciones auxiliares
# --------------------------------------------------------------------------------------

def _fill_fecha_cosecha(fecha_series: pd.Series) -> pd.Series:
    """Convierte a datetime y rellena NaN con la media de fechas válidas."""
    fechas = pd.to_datetime(fecha_series, errors="coerce")
    if fechas.notna().any():
        mean_ts = fechas.dropna().astype("int64").mean()
        mean_fecha = pd.to_datetime(mean_ts)
        return fechas.fillna(mean_fecha)
    # Si todas las fechas son NaN devolvemos la serie tal cual
    return fechas


def _as_numeric(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """Convierte las columnas indicadas a numérico (coerción a NaN si no aplica)."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

# --------------------------------------------------------------------------------------
# Proceso principal
# --------------------------------------------------------------------------------------

def process_carozos(path= path) -> pd.DataFrame:
    """Carga y procesa la hoja *Carozos* siguiendo las reglas solicitadas.

    Parameters
    ----------
    path : str | os.PathLike, optional
        Ruta del archivo Excel. Por defecto busca el archivo en la misma carpeta
        que este script.

    Returns
    -------
    pd.DataFrame
        DataFrame filtrado, con columnas normalizadas y clúster asignado.
    """
    # 1. Cargar datos a partir de la fila 3 (skiprows=2) y columnas A:AP
    df = pd.read_excel(
        path,
        sheet_name=SHEET_NAME,
        usecols=USECOLS,
        skiprows=START_ROW,
        dtype=str,   # primeras conversiones como texto, luego casteamos
    )

    # 2. Filtrar especies válidas
    df = df[df[ESPECIE_COLUMN].isin(ESPECIES_VALIDAS)].copy()

    # 3‑4. Renombrar columnas y seleccionar las relevantes
    rename_map = {"Solidos solubles (%)": "BRIX"}
    df.rename(columns=rename_map, inplace=True)

    selected_columns = (
        [ESPECIE_COLUMN]
        + COLUMNS_FIRMEZA
        + [rename_map[col] if col in rename_map else col for col in COLUMNS_CALIDAD]
        + [DATE_COLUMN]
    )
    df = df[selected_columns]

    # 5. Gestionar columna Fecha cosecha
    df[DATE_COLUMN] = _fill_fecha_cosecha(df[DATE_COLUMN])

    # 6. Conversión a numérico de condiciones y suma
    conditions_cols = COLUMNS_FIRMEZA + ["BRIX", "Acidez (%)"]
    df = _as_numeric(df, conditions_cols)
    df["cond_sum"] = df[conditions_cols].sum(axis=1, skipna=True)

    # 7. Crear 4 clústeres basados en cuartiles
    if df["cond_sum"].notna().nunique() >= 4:
        df["cluster"] = pd.qcut(df["cond_sum"], 4, labels=[1, 2, 3, 4])
    else:
        # Fallback: si hay menos de 4 valores únicos, usamos corte por rangos iguales
        df["cluster"] = pd.cut(df["cond_sum"], bins=4, labels=[1, 2, 3, 4])

    return df

# --------------------------------------------------------------------------------------
# Punto de entrada CLI
# --------------------------------------------------------------------------------------
if __name__ == "__main__":
    processed_df = process_carozos()
    pd.set_option("display.max_columns", None)
    print("\nDataFrame procesado (primeras filas):")
    print(processed_df.head())



DataFrame procesado (primeras filas):
   Especie  Quilla  Hombro  Mejilla 1  Mejilla 2  BRIX  Acidez (%)  \
0  Ciruela     3.0     3.0        2.7        3.0  18.8         NaN   
1  Ciruela     3.0     3.0        3.0        3.7  17.6         NaN   
2  Ciruela     3.5     3.0        3.0        3.7  18.4         NaN   
3  Ciruela     3.5     3.0        5.0        4.7  21.0         NaN   
4  Ciruela     3.5     2.5        3.3        3.0  18.4         NaN   

                  Fecha cosecha  cond_sum cluster  
0 2019-02-25 12:58:01.967213056      30.5       1  
1 2019-02-25 12:58:01.967213056      30.3       1  
2 2019-02-25 12:58:01.967213056      31.6       1  
3 2019-02-25 12:58:01.967213056      37.2       1  
4 2019-02-25 12:58:01.967213056      30.7       1  


In [6]:
processed_df

,Especie,Quilla,Hombro,Mejilla 1,Mejilla 2,BRIX,Acidez (%),Fecha cosecha,cond_sum,cluster
0,Ciruela,3.0,3.0,2.7,3.0,18.8,NaN,2019-02-25 12:58:01.967213056,30.5,1
1,Ciruela,3.0,3.0,3.0,3.7,17.6,NaN,2019-02-25 12:58:01.967213056,30.3,1
2,Ciruela,3.5,3.0,3.0,3.7,18.4,NaN,2019-02-25 12:58:01.967213056,31.6,1
3,Ciruela,3.5,3.0,5.0,4.7,21.0,NaN,2019-02-25 12:58:01.967213056,37.2,1
4,Ciruela,3.5,2.5,3.3,3.0,18.4,NaN,2019-02-25 12:58:01.967213056,30.7,1
...,...,...,...,...,...,...,...,...,...,...
14680,Nectarin,10.0,5.0,9.0,11.0,13.8,NaN,2019-02-25 12:58:01.967213056,48.8,2
14681,Nectarin,15.0,7.0,15.0,15.0,12.0,NaN,2019-02-25 12:58:01.967213056,64.0,2
14682,Nectarin,11.0,7.5,11.7,14.5,13.0,NaN,2019-02-25 12:58:01.967213056,57.7,2
14683,Nectarin,13.0,5.5,12.0,12.3,13.4,NaN,2019-02-25 12:58:01.967213056,56.2,2


In [12]:
# -*- coding: utf-8 -*-
"""
Procesamiento de la hoja «Carozos» del archivo Excel "MAESTRO CAROZOS FINAL COMPLETO CG.xlsx".

**Versión 6 – 24‑jun‑2025**
--------------------------------------------------
### Corrección crítica
Soluciona el **KeyError 'FIRMEZA_PUNTO'** surgido al clasificar Quilla y Hombro.
Ahora `_clasificar_row`:
1. Recibe el nombre real de la columna (`col`).
2. Traduce internamente a la **clave de reglas** correspondiente:
   * `Quilla` o `Hombro` ⇒ `FIRMEZA_PUNTO`
   * `Mejilla 1` o `Mejilla 2` ⇒ `FIRMEZA_MEJ`
   * Las demás columnas se mantienen igual (`BRIX`, `Acidez (%)`).
3. Usa ese rule‑key para obtener la regla y clasificar el **valor contenido en la columna**.

Con esto desaparece el error y cada columna se clasifica contra la tabla correcta.

Uso rápido
~~~~~~~~~~
```python
from procesar_carozos import process_carozos
print(process_carozos().head())
```
"""

from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Tuple, Union

import numpy as np
import pandas as pd

# --------------------------------------------------
# Configuración general
# --------------------------------------------------
path = r"C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\MAESTRO CAROZOS FINAL COMPLETO CG.xlsx"
SHEET_NAME = "CAROZOS"
USECOLS = "A:AP"  # Columnas A … AP
START_ROW = 2       # Cero‑indexado — saltamos las dos primeras filas para que A3 sea la primera

ESPECIE_COLUMN = "Especie"
ESPECIES_VALIDAS = {"Ciruela", "Nectarin"}
DATE_COLUMN = "Fecha cosecha"
WEIGHT_COLS = ["Peso (g)", "Calibre", "Peso"]  # se buscarán en orden

# Columnas de firmeza
COL_FIRMEZA_PUNTO = ("Quilla", "Hombro")
COL_FIRMEZA_MEJILLAS = ("Mejilla 1", "Mejilla 2")
COL_FIRMEZA_TODAS = list(COL_FIRMEZA_PUNTO + COL_FIRMEZA_MEJILLAS)

# Otras columnas
COL_CALIDAD_ORIG = "Solidos solubles (%)"  # será renombrada a BRIX
COL_BRIX = "BRIX"
COL_ACIDEZ = "Acidez (%)"

# --------------------------------------------------
# Reglas CANDY / CHERRY plum  (extraídas del diagrama)
# --------------------------------------------------
PLUM_RULES: Dict[str, Dict[str, List[Tuple[float, float, int]]]] = {
    "candy": {
        COL_BRIX: [(18, np.inf, 1), (16, 18, 2), (14, 16, 3), (-np.inf, 14, 4)],
        "FIRMEZA_PUNTO": [(7, np.inf, 1), (5, 7, 2), (4, 5, 3), (-np.inf, 4, 4)],
        "FIRMEZA_MEJ": [(9, np.inf, 1), (7, 9, 2), (6, 7, 3), (-np.inf, 6, 4)],
        COL_ACIDEZ: [(-np.inf, 0.8, 1), (0.81, 0.88, 2), (0.89, 1.0, 3), (1.0, np.inf, 4)],
    },
    "cherry": {
        COL_BRIX: [(21, np.inf, 1), (18, 21, 2), (15, 18, 3), (-np.inf, 15, 4)],
        "FIRMEZA_PUNTO": [(6, np.inf, 1), (4.5, 6, 2), (3, 4.5, 3), (-np.inf, 3, 4)],
        "FIRMEZA_MEJ": [(8, np.inf, 1), (5, 8, 2), (4, 5, 3), (-np.inf, 4, 4)],
        COL_ACIDEZ: [(-np.inf, 0.8, 1), (0.81, 0.9, 2), (0.91, 1.0, 3), (1.0, np.inf, 4)],
    },
    "unknown": {},
}

# --------------------------------------------------
# Reglas por periodo (para Nectarin u otras especies)
# --------------------------------------------------
DEFAULT_RULES: Dict[str, List[Tuple[float, float, int]]] = {
    COL_BRIX: [(18, np.inf, 1), (15, 18, 2), (12, 15, 3), (-np.inf, 12, 4)],
    COL_ACIDEZ: [(-np.inf, 0.8, 1), (0.8, 0.9, 2), (0.9, 1.0, 3), (1.0, np.inf, 4)],
    "FIRMEZA_PUNTO": [(8, np.inf, 1), (7.9, 8, 2), (7, 7.9, 3), (-np.inf, 7, 4)],
    "FIRMEZA_MEJ": [(13, np.inf, 1), (12, 13, 2), (9, 12, 3), (-np.inf, 9, 4)],
}

PERIOD_RULES: Dict[str, Dict[str, List[Tuple[float, float, int]]]] = {
    p: DEFAULT_RULES.copy() for p in ["muy_temprana", "temprana", "media", "tardia", "sin_fecha"]
}

# --------------------------------------------------
# Periodo de cosecha helper
# --------------------------------------------------

def _harvest_period(fecha: pd.Timestamp) -> str:
    if pd.isna(fecha):
        return "sin_fecha"
    md = (fecha.month, fecha.day)
    if md < (11, 25):
        return "muy_temprana"
    if (11, 25) <= md <= (12, 15):
        return "temprana"
    if (12, 16) <= md <= (2, 15):
        return "media"
    return "tardia"

# --------------------------------------------------
# Funciones auxiliares
# --------------------------------------------------

def _fill_fecha_cosecha(series: pd.Series) -> pd.Series:
    fechas = pd.to_datetime(series, errors="coerce")
    if fechas.notna().any():
        mean_ts = fechas.dropna().astype("int64").mean()
        return fechas.fillna(pd.to_datetime(mean_ts))
    return fechas


def _to_numeric(df: pd.DataFrame, cols: List[str]) -> None:
    for col in cols:
        df[col] = (
            df[col].astype(str).str.replace(",", ".", regex=False).str.replace("\u2212", "-", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")


def _find_weight(row: pd.Series) -> float | None:
    for col in WEIGHT_COLS:
        if col in row and pd.notna(row[col]):
            try:
                return float(str(row[col]).replace(",", "."))
            except ValueError:
                continue
    return None


def _plum_type(row: pd.Series) -> str:
    if row[ESPECIE_COLUMN] != "Ciruela":
        return "non_plum"
    peso = _find_weight(row)
    if peso is None:
        return "unknown"
    return "candy" if peso > 60 else "cherry"


def _rule_key_for_col(col: str) -> str:
    """Mapea las columnas reales al nombre de regla empleado en los dicts."""
    if col in COL_FIRMEZA_PUNTO:
        return "FIRMEZA_PUNTO"
    if col in COL_FIRMEZA_MEJILLAS:
        return "FIRMEZA_MEJ"
    return col  # BRIX, Acidez, etc.


def _clasificar(valor: float, reglas: List[Tuple[float, float, int]]) -> float:
    if pd.isna(valor) or not reglas:
        return np.nan
    for lim_inf, lim_sup, grupo in reglas:
        if lim_inf <= valor < lim_sup:
            return grupo
    return np.nan


def _clasificar_row(row: pd.Series, col: str) -> float:
    rule_key = _rule_key_for_col(col)
    # Try species‑specific (Candy/Cherry) if Ciruela
    if row[ESPECIE_COLUMN] == "Ciruela":
        reglas = PLUM_RULES.get(row["plum_type"], {}).get(rule_key, [])
        if reglas:
            return _clasificar(row[col], reglas)
    # Fallback a reglas por periodo (Nectarin u unknown)
    reglas = PERIOD_RULES.get(row["harvest_period"], {}).get(rule_key, [])
    return _clasificar(row[col], reglas)

# --------------------------------------------------
# Proceso principal
# --------------------------------------------------

def process_carozos(path=path) -> pd.DataFrame:
    """Procesa la hoja «Carozos» con reglas Candy/Cherry y periodos."""

    # 1 | Cargar datos
    df = pd.read_excel(path, sheet_name=SHEET_NAME, usecols=USECOLS, skiprows=START_ROW, dtype=str)

    # 2 | Filtrar especies válidas
    df = df[df[ESPECIE_COLUMN].isin(ESPECIES_VALIDAS)].copy()

    # 3 | Normalizar nombres de columnas
    df.rename(columns={COL_CALIDAD_ORIG: COL_BRIX}, inplace=True)

    # 4 | Fecha y harvest period
    df[DATE_COLUMN] = _fill_fecha_cosecha(df[DATE_COLUMN])
    df["harvest_period"] = df[DATE_COLUMN].apply(_harvest_period)

    # 5 | Plum type
    df["plum_type"] = df.apply(_plum_type, axis=1)

    # 6 | Conversión numérica
    numeric_cols = COL_FIRMEZA_TODAS + [COL_BRIX, COL_ACIDEZ] + [c for c in WEIGHT_COLS if c in df.columns]
    _to_numeric(df, numeric_cols)

    # 7 | Clasificación
    cols_to_classify = COL_FIRMEZA_TODAS + [COL_BRIX, COL_ACIDEZ]
    for col in cols_to_classify:
        df[f"grp_{col.replace(' ', '_')}"] = df.apply(lambda r, c=col: _clasificar_row(r, c), axis=1)

    # 8 | Suma y clúster
    grp_cols = [c for c in df.columns if c.startswith("grp_")]
    df["cond_sum"] = df[grp_cols].sum(axis=1)
    if df["cond_sum"].notna().nunique() >= 4:
        df["cluster"] = pd.qcut(df["cond_sum"], 4, labels=[1, 2, 3, 4])
    else:
        df["cluster"] = pd.cut(df["cond_sum"], bins=4, labels=[1, 2, 3, 4])

    return df

# --------------------------------------------------
# CLI
# --------------------------------------------------
if __name__ == "__main__":
    pd.set_option("display.max_columns", None)
    df = process_carozos()
    print(df.head())


   Especie Variedad     PMG        Localidad  Temporada  \
0  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017   
1  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017   
2  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017   
3  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017   
4  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017   

                  Fecha cosecha     Fecha evaluación Periodo de almacenaje  \
0 2019-02-25 12:58:01.967213056  2017-01-12 00:00:00               Cosecha   
1 2019-02-25 12:58:01.967213056  2017-01-12 00:00:00               Cosecha   
2 2019-02-25 12:58:01.967213056  2017-01-12 00:00:00               Cosecha   
3 2019-02-25 12:58:01.967213056  2017-01-12 00:00:00               Cosecha   
4 2019-02-25 12:58:01.967213056  2017-01-12 00:00:00               Cosecha   

  Raleo (frutos/pl) Planta Perimetro Rendimiento Repetición 0-30 30-50 50-75  \
0               NaN    NaN       NaN         NaN        NaN  NaN   NaN   NaN   
1               

In [14]:
df.to_excel(r'C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\output.xlsx')

C:\Users\gonzalo.rojas\AppData\Local\Temp\ipykernel_25552\3133697872.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.2' currently installed).
  df.to_excel(r'C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\output.xlsx')


In [5]:
# -*- coding: utf-8 -*-
"""
Procesamiento de la hoja «Carozos» del archivo Excel "MAESTRO CAROZOS FINAL COMPLETO CG.xlsx".

Versión 7 – 25‑jun‑2025
--------------------------------------------------
Novedades claves
----------------
1. **Dos sub‑tipos de cherry plum** («small» ≤ 45 g y «mid» 46‑60 g) con reglas
   independientes y fácilmente editables.
2. **Nuevos umbrales Candy / Cherry** (Brix, Firmeza, Acidez, Productividad)
   exactamente como en el flujograma.
3. **Color de pulpa** («Amarilla» ∣ «Blanca») para nectarines
   + reglas específicas por periodo de cosecha (muy temprana, temprana, tardía).
4. **Firmeza punto débil**  
   = *valor mínimo más frecuente* entre Quilla, Hombro, Mejilla 1 y 2,
   calculado en el **promedio del grupo Variedad + Fruto**.
5. **Relleno de nulos**: si una muestra carece de X, se toma el valor de la
   **muestra 1** del mismo grupo.
6. **Clúster doble**  
   - `cluster_row`  → cada registro individual  
   - `cluster_grp`  → promedio del grupo Variedad + Fruto
"""

from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Union, Sequence, Mapping

# --------------------------------------------------
# Configuración general
# --------------------------------------------------
FILE = Path(
    r"C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF"
    r"\___17__segementación especies\files\MAESTRO CAROZOS FINAL COMPLETO CG.xlsx"
)
SHEET_NAME = "CAROZOS"
USECOLS = "A:AP"
START_ROW = 2               # saltar filas informativas

ESPECIE_COLUMN = "Especie"
DATE_COLUMN = "Fecha cosecha"
COLOR_COLUMN = "Color de pulpa"           # solo existe en Nectarin

VAR_COLUMN   = "Variedad"                 # llave para agrupaciones
FRUTO_COLUMN = "Fruto (n°)"

ESPECIES_VALIDAS = {"Ciruela", "Nectarin"}

# ---------------------------------------------------------------------------
# Columnas físicas
# ---------------------------------------------------------------------------
WEIGHT_COLS = ("Peso (g)", "Calibre", "Peso")
COL_FIRMEZA_PUNTO    = ("Quilla", "Hombro")
COL_FIRMEZA_MEJILLAS = ("Mejilla 1", "Mejilla 2")
COL_FIRMEZA_ALL      = list(COL_FIRMEZA_PUNTO + COL_FIRMEZA_MEJILLAS)

COL_ORIG_BRIX = "Solidos solubles (%)"
COL_BRIX      = "BRIX"
COL_ACIDEZ    = "Acidez (%)"
#COL_PROD      = "Productividad (Ton)"

NUMERIC_COLS = (
    COL_FIRMEZA_ALL
    + [COL_BRIX, COL_ACIDEZ]#, COL_PROD]
    + [c for c in WEIGHT_COLS]
)

# ---------------------------------------------------------------------------
# Tabla de reglas ― Ciruela --------------------------------------------------
#   Cada lista: [(mín, máx, grupo), …]
#   Límite inf. incluido, sup. excluido  → [mín, máx)
# ---------------------------------------------------------------------------
PLUM_RULES: Dict[str, Dict[str, List[Tuple[float, float, int]]]] = {
    # CANDY PLUM  Calibre > 60 g
    "candy": {
        COL_BRIX:      [(18.0,  np.inf, 1), (16.0, 18.0, 2), (14.0, 16.0, 3), (-np.inf, 14.0, 4)],
        "FIRMEZA_PUNTO": [(7.0,  np.inf, 1), (5.0,  7.0, 2), (4.0,  5.0, 3), (-np.inf, 4.0, 4)],
        "FIRMEZA_MEJ": [(9.0,  np.inf, 1), (7.0,  9.0, 2), (6.0,  7.0, 3), (-np.inf, 6.0, 4)],
        COL_ACIDEZ:    [(-np.inf, 0.60, 1), (0.60, 0.81, 2), (0.81, 1.00, 3), (1.00, np.inf, 4)]#,
       # COL_PROD:      [(40, np.inf, 1), (35, 40, 2), (25, 35, 3), (-np.inf, 25, 4)],
    },
    # CHERRY PLUM mid (46‑60 g)
    "cherry_mid": {
        COL_BRIX:      [(21.0, np.inf, 1), (18.0, 21.0, 2), (15.0, 18.0, 3), (-np.inf, 15.0, 4)],
        "FIRMEZA_PUNTO": [(6.0, np.inf, 1), (4.5,  6.0, 2), (3.0,  4.5, 3), (-np.inf, 3.0, 4)],
        "FIRMEZA_MEJ": [(8.0, np.inf, 1), (5.0,  8.0, 2), (4.0,  5.0, 3), (-np.inf, 4.0, 4)],
        COL_ACIDEZ:    [(-np.inf, 0.60, 1), (0.60, 0.81, 2), (0.81, 1.00, 3), (1.00, np.inf, 4)]#,
       # COL_PROD:      [(30, np.inf, 1), (20, 30, 2), (15, 20, 3), (-np.inf, 15, 4)],
    },
    # CHERRY PLUM small (≤ 45 g)  – por ahora mismo set que ‘mid’; cámbialo si
    #   el comité técnico define otros valores
    "cherry_small": {},   # se hereda dinámicamente más abajo
}

# hereda reglas mid si no se redefine
PLUM_RULES["cherry_small"] = PLUM_RULES["cherry_mid"].copy()

# ---------------------------------------------------------------------------
# Tabla de reglas ― Nectarin ------------------------------------------------
#   Se desdobla por color de pulpa y periodo de cosecha
# ---------------------------------------------------------------------------
def _mk_nec_rules(
    brix1: float, brix2: float, brix3: float,
    mej_1: float, mej_2: float,
) -> Dict[str, List[Tuple[float, float, int]]]:
    """Helper: genera tabla estándar Nectarín."""
    return {
        COL_BRIX: [(brix1, np.inf, 1), (brix2, brix1, 2), (brix3, brix2, 3), (-np.inf, brix3, 4)],
        "FIRMEZA_PUNTO": [(9.0, np.inf, 1), (8.0, 9.0, 2), (7.0, 8.0, 3), (-np.inf, 7.0, 4)],
        "FIRMEZA_MEJ": [(mej_1, np.inf, 1), (mej_2, mej_1, 2), (9.0, mej_2, 3), (-np.inf, 9.0, 4)],
        COL_ACIDEZ: [(-np.inf, 0.60, 1), (0.60, 0.81, 2), (0.81, 1.00, 3), (1.00, np.inf, 4)],
    }

NECT_RULES: Dict[str, Dict[str, Dict[str, List[Tuple[float, float, int]]]]] = {
    # Color ↦ Periodo ↦ Reglas
    "amarilla": {
        "muy_temprana": _mk_nec_rules(13.0, 10.0, 9.0, 14.0, 12.0),
        "temprana":      _mk_nec_rules(13.0, 10.0, 9.0, 14.0, 12.0),
        "tardia":        _mk_nec_rules(14.0, 12.0, 10.0, 14.0, 12.0),
    },
    "blanca": {
        "muy_temprana": _mk_nec_rules(13.0, 10.0, 9.0, 13.0, 11.0),
        "temprana":      _mk_nec_rules(13.0, 10.0, 9.0, 13.0, 11.0),
        "tardia":        _mk_nec_rules(14.0, 12.0, 10.0, 13.0, 11.0),
    },
}

# Periodos aceptados en Nectarín; si cae entre 16‑dic y 15‑feb lo normalizamos a
# «media» (se comporta como «temprana»)
PERIOD_MAP = {
    "muy_temprana": "muy_temprana",
    "temprana": "temprana",
    "media": "temprana",
    "tardia": "tardia",
    "sin_fecha": "temprana",
}

# --------------------------------------------------
# Helpers fecha
# --------------------------------------------------
def _harvest_period(ts: pd.Timestamp | float | str) -> str:
    ts = pd.to_datetime(ts, errors="coerce")
    if pd.isna(ts):
        return "sin_fecha"
    m, d = ts.month, ts.day
    if (m, d) < (11, 25):
        return "muy_temprana"
    if (11, 25) <= (m, d) <= (12, 15):
        return "temprana"
    if (12, 16) <= (m, d) <= (2, 15):
        return "media"
    return "tardia"

# --------------------------------------------------
# Conversions & fills
# --------------------------------------------------
def _to_numeric(df: pd.DataFrame, cols: Sequence[str]) -> None:
    for c in cols:
        if c not in df.columns:
            continue
        df[c] = (
            df[c]
            .astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace("\u2212", "-", regex=False)
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

def _first_sample_fill(group: pd.DataFrame, cols: Sequence[str]) -> pd.DataFrame:
    """Rellena NaNs del grupo con el valor de la primera muestra."""
    first = group.iloc[0]
    for c in cols:
        if c in group:
            group[c] = group[c].fillna(first[c])
    return group

def _weight_value(row: pd.Series) -> float | None:
    for c in WEIGHT_COLS:
        if c in row and pd.notna(row[c]):
            try:
                return float(str(row[c]).replace(",", "."))
            except ValueError:
                continue
    return None

# --------------------------------------------------
# Detección de tipo de ciruela
# --------------------------------------------------
def _plum_subtype(row: pd.Series) -> str:
    """candy / cherry_mid / cherry_small / unknown / non_plum"""
    if row[ESPECIE_COLUMN] != "Ciruela":
        return "non_plum"
    peso = _weight_value(row)
    if peso is None:
        return "unknown"
    if peso > 60:
        return "candy"
    if peso > 45:
        return "cherry_mid"
    return "cherry_small"

# --------------------------------------------------
# Firmeza punto débil (mínimo más frecuente)
# --------------------------------------------------
def _fpd_from_group(grp: pd.DataFrame) -> float | None:
    mean_vals = grp[COL_FIRMEZA_ALL].mean()
    # frecuencia del valor mínimo
    min_val = mean_vals.min()
    if pd.isna(min_val):
        return np.nan
    # romper empates tomando el primer campo izquierda‑derecha
    return float(min_val)

# --------------------------------------------------
# Clasificador genérico
# --------------------------------------------------
def _rule_key(col: str) -> str:
    if col in COL_FIRMEZA_PUNTO or col == "Firmeza punto débil":
        return "FIRMEZA_PUNTO"
    if col in COL_FIRMEZA_MEJILLAS:
        return "FIRMEZA_MEJ"
    return col

def _classify_value(val: float, rules: List[Tuple[float, float, int]]) -> float:
    if pd.isna(val) or not rules:
        return np.nan
    for lo, hi, grp in rules:
        if lo <= val < hi:
            return grp
    return np.nan

def _classify_row(row: pd.Series, col: str) -> float:
    key = _rule_key(col)
    # CIRUELA ---------------------------------------------------------------
    if row[ESPECIE_COLUMN] == "Ciruela":
        rules = PLUM_RULES.get(row["plum_subtype"], {}).get(key, [])
        if rules:
            return _classify_value(row[col], rules)
    # NECTARIN --------------------------------------------------------------
    if row[ESPECIE_COLUMN] == "Nectarin":
        color = str(row[COLOR_COLUMN]).strip().lower() or "amarilla"
        color = "blanca" if color.startswith("blanc") else "amarilla"
        period = PERIOD_MAP[row["harvest_period"]]
        rules = NECT_RULES[color][period].get(key, [])
        return _classify_value(row[col], rules)
    # Fallback
    return np.nan

# --------------------------------------------------
# Pipeline principal
# --------------------------------------------------
def process_carozos(file: Union[str, Path] = FILE) -> pd.DataFrame:
    df = pd.read_excel(
        file, sheet_name=SHEET_NAME, usecols=USECOLS,
        skiprows=START_ROW, dtype=str
    )

    # ------------ filtros & renombres -------------------------------------
    df = df[df[ESPECIE_COLUMN].isin(ESPECIES_VALIDAS)].copy()
    df.rename(columns={COL_ORIG_BRIX: COL_BRIX}, inplace=True)
    if COLOR_COLUMN not in df.columns:
        df[COLOR_COLUMN] = "Amarilla"

    # ------------ tipos y periodos ----------------------------------------
    df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], errors="coerce")
    df["harvest_period"] = df[DATE_COLUMN].apply(_harvest_period)
    df["plum_subtype"] = df.apply(_plum_subtype, axis=1)

    # ------------ numéricos ------------------------------------------------
    _to_numeric(df, NUMERIC_COLS)

    # ------------ agrupación Variedad + Fruto -----------------------------
    grp_keys = [VAR_COLUMN, FRUTO_COLUMN]
    # Firmeza punto débil grupal
    grp_fpd = (
        df.groupby(grp_keys, dropna=False)
          .apply(_fpd_from_group)
          .rename("Firmeza punto débil")
          .reset_index()
    )
    df = df.merge(grp_fpd, on=grp_keys, how="left")

    # Relleno de nulos por primera muestra
    df = (
        df.groupby(grp_keys, dropna=False, group_keys=False)
          .apply(_first_sample_fill, NUMERIC_COLS + ["Firmeza punto débil"])
    )

    # ------------ clasificación -------------------------------------------
    cols_to_classify = ["Firmeza punto débil"] + [COL_BRIX, COL_ACIDEZ]#, COL_PROD]
    for col in cols_to_classify:
        out = f"grp_{col.replace(' ', '_')}"
        df[out] = df.apply(lambda r, c=col: _classify_row(r, c), axis=1)

    # ------------ cluster individual --------------------------------------
    grp_cols = [c for c in df.columns if c.startswith("grp_")]
    df["cond_sum"] = df[grp_cols].sum(axis=1, min_count=1)
    if df["cond_sum"].notna().nunique() >= 4:
        df["cluster_row"] = pd.qcut(df["cond_sum"], 4, labels=[1, 2, 3, 4])
    else:
        df["cluster_row"] = pd.cut(df["cond_sum"], 4, labels=[1, 2, 3, 4])

    # ------------ cluster grupal (promedio) -------------------------------
    grp_cond = (
        df.groupby(grp_keys, dropna=False)["cond_sum"]
          .mean()
          .rename("cond_sum_grp")
          .reset_index()
    )
    df = df.merge(grp_cond, on=grp_keys, how="left")

    if grp_cond["cond_sum_grp"].notna().nunique() >= 4:
        bins = pd.qcut(grp_cond["cond_sum_grp"], 4, labels=[1, 2, 3, 4])
    else:
        bins = pd.cut(grp_cond["cond_sum_grp"], 4, labels=[1, 2, 3, 4])
    grp_cond["cluster_grp"] = bins

    df = df.merge(grp_cond[grp_keys + ["cluster_grp"]], on=grp_keys, how="left")

    return df

# ------------------------------ CLI ----------------------------------------
if __name__ == "__main__":
    pd.set_option("display.max_columns", None)
    df = process_carozos()
    print(process_carozos().head())


C:\Users\gonzalo.rojas\AppData\Local\Temp\ipykernel_36892\2007667247.py:279: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(grp_keys, dropna=False)
C:\Users\gonzalo.rojas\AppData\Local\Temp\ipykernel_36892\2007667247.py:288: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(grp_keys, dropna=False, group_keys=False)
C:\Users\gonzalo.rojas\AppData\Local\Temp\ipykernel_36892\2007667247.py:279:

   Especie Variedad     PMG        Localidad  Temporada Fecha cosecha  \
0  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017           NaT   
1  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017           NaT   
2  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017           NaT   
3  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017           NaT   
4  Ciruela   CI-181  Zaiger  Vivero Requinoa  2016-2017           NaT   

      Fecha evaluación Periodo de almacenaje Raleo (frutos/pl) Planta  \
0  2017-01-12 00:00:00               Cosecha               NaN    NaN   
1  2017-01-12 00:00:00               Cosecha               NaN    NaN   
2  2017-01-12 00:00:00               Cosecha               NaN    NaN   
3  2017-01-12 00:00:00               Cosecha               NaN    NaN   
4  2017-01-12 00:00:00               Cosecha               NaN    NaN   

  Perimetro Rendimiento Repetición 0-30 30-50 50-75 75-100 Total Verde Crema  \
0       NaN         NaN        NaN  NaN   

In [6]:
df.to_excel(r'C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\output2.xlsx')

C:\Users\gonzalo.rojas\AppData\Local\Temp\ipykernel_36892\1558643936.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.2' currently installed).
  df.to_excel(r'C:\Users\gonzalo.rojas\OneDrive - GARCES FRUIT\Escritorio\LAB DATOS GF\___17__segementación especies\files\output2.xlsx')
